# Challenge: Data visualization

## Importación

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##📌 Extracción

In [3]:
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


##🔧 Transformación

In [4]:
df_customer = pd.json_normalize(df['customer']) #Normalizando las columnas
df_phone = pd.json_normalize(df['phone']) #Con diccionarios/json
df_internet = pd.json_normalize(df['internet'])
df_account = pd.json_normalize(df['account'])

df_charges_monthly = df_account['Charges.Monthly'].rename('ChargeMonthly') #Asignando a un df cada columna
df_charges_total = df_account['Charges.Total'].rename('ChargeTotal')  #anidada de los cargos

df_account.drop(columns=['Charges.Monthly','Charges.Total'], inplace=True)
df_account.head() #Eliminando las columnas reasignadas

,Contract,PaperlessBilling,PaymentMethod
0,One year,Yes,Mailed check
1,Month-to-month,No,Mailed check
2,Month-to-month,Yes,Electronic check
3,Month-to-month,Yes,Electronic check
4,Month-to-month,Yes,Mailed check


In [5]:
# Uniendo los dataframes ya normalizados
df_norm = pd.concat([df['Churn'], df_customer, df_phone,df_internet,df_account,df_charges_monthly,df_charges_total], axis=1)
df_norm.to_csv('df_norm.csv', index=False)
df_norm.info()

df_norm['ChargeTotal'] = pd.to_numeric(df_norm['ChargeTotal'], errors='coerce')
df_norm['ChargeDaily'] = df_norm['ChargeMonthly'] / 30
df_norm['Churn'] = df_norm['Churn'].fillna('No') #Rellenando datos vacios
df_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7267 non-null   object 
 1   gender            7267 non-null   object 
 2   SeniorCitizen     7267 non-null   int64  
 3   Partner           7267 non-null   object 
 4   Dependents        7267 non-null   object 
 5   tenure            7267 non-null   int64  
 6   PhoneService      7267 non-null   object 
 7   MultipleLines     7267 non-null   object 
 8   InternetService   7267 non-null   object 
 9   OnlineSecurity    7267 non-null   object 
 10  OnlineBackup      7267 non-null   object 
 11  DeviceProtection  7267 non-null   object 
 12  TechSupport       7267 non-null   object 
 13  StreamingTV       7267 non-null   object 
 14  StreamingMovies   7267 non-null   object 
 15  Contract          7267 non-null   object 
 16  PaperlessBilling  7267 non-null   object 


Imprimiendo una muestra aleatoria de 5 registros del _dataframe_

In [6]:
df_norm.sample(5)

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,ChargeMonthly,ChargeTotal,ChargeDaily
6242,No,Female,0,No,No,41,Yes,Yes,DSL,Yes,...,Yes,Yes,No,Yes,Month-to-month,Yes,Bank transfer (automatic),74.65,3090.65,2.488333
458,Yes,Male,0,Yes,Yes,21,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,104.40,2157.95,3.480000
6982,No,Female,0,No,No,4,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Credit card (automatic),20.45,82.85,0.681667
1027,Yes,Male,0,Yes,No,70,Yes,Yes,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,One year,No,Credit card (automatic),115.65,7968.85,3.855000
7120,No,Female,0,Yes,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),75.80,1615.10,2.526667


Imprimiendo datos estadísticos del _dataframe_

In [7]:
df_norm.describe()

,SeniorCitizen,tenure,ChargeMonthly,ChargeTotal,ChargeDaily
count,7267.000000,7267.000000,7267.000000,7256.000000,7267.000000
mean,0.162653,32.346498,64.720098,2280.634213,2.157337
std,0.369074,24.571773,30.129572,2268.632997,1.004319
min,0.000000,0.000000,18.250000,18.800000,0.608333
25%,0.000000,9.000000,35.425000,400.225000,1.180833
50%,0.000000,29.000000,70.300000,1391.000000,2.343333
75%,0.000000,55.000000,89.875000,3785.300000,2.995833
max,1.000000,72.000000,118.750000,8684.800000,3.958333


##📊 Carga y análisis

##📄Informe final